In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
tf.reset_default_graph()

In [2]:
#Preprocessing 
df=pd.read_csv('HeartDisease.csv')
num_sapmles=365
shuffle_df=df.sample(frac=1)
outputs=shuffle_df[['num']]
no_samples=outputs.shape[0]
print(no_samples)
inputs=shuffle_df.drop(['num','thalach','exang','oldpeak','Place','thalach','Age','Sex','ID'],axis=1)
va=outputs.tail(1).values
inputs['trestbps']=inputs['trestbps'].fillna(np.mean(inputs['trestbps']))
inputs['fbs']=inputs['fbs'].fillna(np.mean(inputs['fbs']))
inputs['cp']=inputs['cp'].fillna(np.mean(inputs['cp']))
inputs['chol']=inputs['chol'].fillna(np.mean(inputs['chol']))
inputs['restecg']=inputs['restecg'].fillna(np.mean(inputs['restecg']))

#scale data

inputs=inputs/np.amax(inputs,axis=0)
test_x=inputs.tail(92).values
test_y=outputs.tail(92).values
test_x=np.float32(test_x)
test_y=np.float32(test_y)
inputs=inputs.head(num_sapmles).values
outputs=outputs.head(num_sapmles).values

inputs=np.float32(inputs)
outputs=np.float32(outputs)
print(test_x.shape)

457
(92, 5)


In [3]:
x=tf.placeholder(tf.float32,[None,5])
y=tf.placeholder(tf.float32,[None,1])

In [4]:
def buildNN(numL,neurPerLayer):
    num=1
    wlst=[]
    blst=[]
    for i in range(numL):
        weight='weight'+str(num)
        bias='bias'+str(num)
        wlst.append(weight)
        blst.append(bias)
        num+=1
    #print(wlst)
    nl=0
    dictLayers={}
    layer='layer'+str(nl)
    dictLayers[layer]=[tf.Variable(tf.random_normal([5,neurPerLayer]),name=wlst[0]),tf.Variable(tf.random_normal([neurPerLayer]),name=blst[0])]
    for i in range(1,numL-1,1):
        nl=i
        layer='layer'+str(nl)
        dictLayers[layer]=[tf.Variable(tf.random_normal([neurPerLayer,neurPerLayer]),name=wlst[i]),tf.Variable(tf.random_normal([neurPerLayer]),name=blst[i])]
        i+=1
    layer='layer'+str(numL-1)
    dictLayers[layer]=[tf.Variable(tf.random_normal([neurPerLayer,1]),name=wlst[numL-1]),tf.Variable(tf.random_normal([1]),name=blst[numL-1])]
    saver = tf.train.Saver()  
        
        
    return dictLayers,saver

    



In [5]:
neurPerLayer=50
numL=40
dictLayer,saver=buildNN(numL,neurPerLayer)

In [6]:
def Forward(dictLayer,inputs):
    lst=[]
    list_all_outs=[]
    o1=tf.nn.sigmoid(tf.add(tf.matmul(inputs,dictLayer['layer0'][0]),dictLayer['layer0'][1]))
    lst.append(o1)
    list_all_outs.append(o1)
    del(dictLayer['layer0'])
    print(list_all_outs)
    for k in dictLayer:
        o=tf.nn.sigmoid(tf.add(tf.matmul(lst[0],dictLayer[k][0]),dictLayer[k][1]))
        print(o)
        list_all_outs.append(o)
        del(lst[0])
        lst.append(o)  
    
    return list_all_outs[-1]

In [7]:
def Train(data):
    pred=Forward(dictLayer,inputs)
    #### init 
    cross = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=pred)
    cost = tf.reduce_mean(cross)
    optimizer=tf.train.MomentumOptimizer(learning_rate=0.01,momentum=0.93).minimize(cost)
    correct_prediction = tf.equal(tf.round(pred),y, name="correct_prediction")
    #tf.summary.scalar('Loss', optimizer)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name="accuracy")
    init=tf.global_variables_initializer()
    test_feed = {x:test_x}
    epl=[]
    lsl=[]
    with tf.Session() as sess:
        writer = tf.summary.FileWriter('./logs', sess.graph)
        sess.run(init)
        for epoch in range(30000) :
            c=sess.run(optimizer,feed_dict={x:data,y:outputs}) 
            cc=sess.run(cost,feed_dict={x:data,y:outputs})
            if epoch % 1000 == 0 :
                print('epoch : '+str(epoch)+'\t Loss :',(cc))    
            #if epoch % 100 ==0 :
            epl.append(epoch)            
            lsl.append(cc)
                
        saver.save(sess, "./hrt.ckpt")
       
     ##########################
    
    return epl,lsl    

epl,lsl=Train(inputs) 


[<tf.Tensor 'Sigmoid:0' shape=(365, 50) dtype=float32>]
Tensor("Sigmoid_1:0", shape=(365, 50), dtype=float32)
Tensor("Sigmoid_2:0", shape=(365, 50), dtype=float32)
Tensor("Sigmoid_3:0", shape=(365, 50), dtype=float32)
Tensor("Sigmoid_4:0", shape=(365, 50), dtype=float32)
Tensor("Sigmoid_5:0", shape=(365, 50), dtype=float32)
Tensor("Sigmoid_6:0", shape=(365, 50), dtype=float32)
Tensor("Sigmoid_7:0", shape=(365, 50), dtype=float32)
Tensor("Sigmoid_8:0", shape=(365, 50), dtype=float32)
Tensor("Sigmoid_9:0", shape=(365, 50), dtype=float32)
Tensor("Sigmoid_10:0", shape=(365, 50), dtype=float32)
Tensor("Sigmoid_11:0", shape=(365, 50), dtype=float32)
Tensor("Sigmoid_12:0", shape=(365, 50), dtype=float32)
Tensor("Sigmoid_13:0", shape=(365, 50), dtype=float32)
Tensor("Sigmoid_14:0", shape=(365, 50), dtype=float32)
Tensor("Sigmoid_15:0", shape=(365, 50), dtype=float32)
Tensor("Sigmoid_16:0", shape=(365, 50), dtype=float32)
Tensor("Sigmoid_17:0", shape=(365, 50), dtype=float32)
Tensor("Sigmoid_18

KeyboardInterrupt: 

In [8]:
mepl=epl
mlsl=lsl

NameError: name 'epl' is not defined

In [9]:
len(lsl)

NameError: name 'lsl' is not defined

In [10]:
plt.plot(epl,lsl,'g',label='moment')
plt.legend(loc='upper right')
plt.show()

NameError: name 'epl' is not defined

In [174]:
tf.reset_default_graph()
imported_meta = tf.train.import_meta_graph("./hrt.ckpt.meta") 
with tf.Session() as sess:
    imported_meta.restore(sess, tf.train.latest_checkpoint('./'))
    num=1
    wlst=[]
    blst=[]
    for i in range(0,numL,1):
        weight='weight'+str(num)+':0'
        bias='bias'+str(num)+':0'
        wlst.append(weight)
        blst.append(bias)
        num+=1
    pr_dict={}
    for i in range(0,numL,1):
        layer='layer'+str(i)
        pr_dict[layer]=[sess.run(wlst[i]),sess.run(blst[i])]
    outt=Forward(pr_dict,test_x)
    outt=sess.run(outt)
    print(outt)        
    
    

INFO:tensorflow:Restoring parameters from ./hrt.ckpt
[<tf.Tensor 'Sigmoid_10:0' shape=(92, 50) dtype=float32>]
Tensor("Sigmoid_11:0", shape=(92, 50), dtype=float32)
Tensor("Sigmoid_12:0", shape=(92, 50), dtype=float32)
Tensor("Sigmoid_13:0", shape=(92, 50), dtype=float32)
Tensor("Sigmoid_14:0", shape=(92, 50), dtype=float32)
Tensor("Sigmoid_15:0", shape=(92, 50), dtype=float32)
Tensor("Sigmoid_16:0", shape=(92, 50), dtype=float32)
Tensor("Sigmoid_17:0", shape=(92, 50), dtype=float32)
Tensor("Sigmoid_18:0", shape=(92, 50), dtype=float32)
Tensor("Sigmoid_19:0", shape=(92, 1), dtype=float32)
[[0.9655231 ]
 [0.00236439]
 [0.00246657]
 [0.00251282]
 [0.00735717]
 [0.00654293]
 [0.986225  ]
 [0.9869581 ]
 [0.02735054]
 [0.00246097]
 [0.00857957]
 [0.00857957]
 [0.00397506]
 [0.9845163 ]
 [0.00271461]
 [0.00233278]
 [0.0060627 ]
 [0.0024027 ]
 [0.114151  ]
 [0.9783136 ]
 [0.9856596 ]
 [0.00237829]
 [0.00388449]
 [0.00553599]
 [0.71562237]
 [0.81079346]
 [0.00681278]
 [0.9823797 ]
 [0.00492319

In [175]:
outt

array([[0.9655231 ],
       [0.00236439],
       [0.00246657],
       [0.00251282],
       [0.00735717],
       [0.00654293],
       [0.986225  ],
       [0.9869581 ],
       [0.02735054],
       [0.00246097],
       [0.00857957],
       [0.00857957],
       [0.00397506],
       [0.9845163 ],
       [0.00271461],
       [0.00233278],
       [0.0060627 ],
       [0.0024027 ],
       [0.114151  ],
       [0.9783136 ],
       [0.9856596 ],
       [0.00237829],
       [0.00388449],
       [0.00553599],
       [0.71562237],
       [0.81079346],
       [0.00681278],
       [0.9823797 ],
       [0.00492319],
       [0.00288012],
       [0.00263885],
       [0.0029173 ],
       [0.98604506],
       [0.00238229],
       [0.70701265],
       [0.7432084 ],
       [0.00452755],
       [0.88976365],
       [0.00291868],
       [0.00731126],
       [0.84397113],
       [0.03398726],
       [0.00231225],
       [0.8184451 ],
       [0.0023812 ],
       [0.45981514],
       [0.9586767 ],
       [0.033

In [176]:
clf = np.where(outt<0.5,0,1)


In [177]:
clf

array([[1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1],
       [0],
       [1],
    

In [178]:
accrc = np.sum(clf==test_y) / len(test_y)
print('Accuracy : '+str(accrc*100)+' %')

Accuracy : 72.82608695652173 %


In [179]:
test_y

array([[1.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [1.],

In [181]:
lossfile=open('sgd_loss.txt','w')
for i in mlsl:
    lossfile.write(str(i))
    lossfile.write('\n')
    
lossfile.close()    

    

In [745]:
len(lsl)

30000